# 🔄 Step 4: Build Multi-Agent Reasoning Pipeline with LangGraph

We define agent nodes and wire them into a directed reasoning graph.
Each node:
- Accepts a shared state
- Adds/modifies entries in the state
- Passes it to the next node

In [1]:
!pip install langgraph

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, List

class AgentState(TypedDict):
    user_question: str
    rewritten_question: str
    retrieved_docs: List[str]
    fact_check: str
    answer: str

In [3]:
def rewrite_node(state: AgentState) -> AgentState:
    rewritten = query_rewriter_agent(state["user_question"])
    return {**state, "rewritten_question": rewritten}

def retrieve_node(state: AgentState) -> AgentState:
    docs = retrieval_agent(state["rewritten_question"])
    return {**state, "retrieved_docs": docs}

def factcheck_node(state: AgentState) -> AgentState:
    check = fact_checker_agent(state["user_question"], state["retrieved_docs"])
    return {**state, "fact_check": check}

def synthesize_node(state: AgentState) -> AgentState:
    final = synthesis_agent(state["user_question"], state["retrieved_docs"])
    return {**state, "answer": final}

In [4]:
graph = StateGraph(AgentState)

graph.add_node("rewrite", rewrite_node)
graph.add_node("retrieve", retrieve_node)
graph.add_node("factcheck", factcheck_node)
graph.add_node("synthesize", synthesize_node)

graph.set_entry_point("rewrite")
graph.add_edge("rewrite", "retrieve")
graph.add_edge("retrieve", "factcheck")
graph.add_edge("factcheck", "synthesize")
graph.add_edge("synthesize", END)

rag_graph = graph.compile()

In [5]:
user_input = "When did Isaac Newton publish the Principia?"

initial_state = {
    "user_question": user_input
}

final_state = rag_graph.invoke(initial_state)

print("🔁 Rewritten Query:\n", final_state["rewritten_question"])
print("\n📚 Top Retrieved Doc:\n", final_state["retrieved_docs"][0][:500])
print("\n✅ Fact Check:\n", final_state["fact_check"])
print("\n🧠 Final Answer:\n", final_state["answer"])

NameError: name 'query_rewriter_agent' is not defined